In [ ]:
from dotenv import load_dotenv
import json
import os
from datetime import datetime as dt, timezone
import time
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentRunStream,
    BingGroundingTool
    FileSearchTool,
    CodeInterpreterTool,
    FunctionTool,
    FilePurpose,
    MessageImageFileContent,
    RunStatus,
    SubmitToolOutputsAction,
    AgentEventHandler,
    MessageDeltaChunk,
    MessageDeltaTextContent,
    ThreadMessage,
    ThreadRun,
    ToolSet,
    RunStepDeltaChunk,
    RunStepDeltaToolCallObject,
    RunStepDeltaMessageCreation,
    RunStepDeltaCodeInterpreterImageOutput,
    RunStepDeltaCodeInterpreterLogOutput,
)
from azure.identity import DefaultAzureCredential
from pathlib import Path
from advanced_reasoning import advanced_reasoning

load_dotenv()

# Initialize the AI Project Client
project_client = AIProjectClient(
    endpoint=os.environ["ENDPOINT"],
    subscription_id=os.environ["SUBSCRIPTION_ID"],
    resource_group_name=os.environ["RESOURCE_GROUP_NAME"],
    project_name=os.environ["PROJECT_NAME"],
    credential=DefaultAzureCredential(),
)

bing_connection = project_client.connections.get(
    connection_name=os.getenv("BING_CONNECTION_NAME")
)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=bing_connection)

# Get current UTC date and time in a readable format
current_datetime = dt.now(timezone.utc).strftime('%B %d, %Y')

# Define the updated instructions
instructions = f"""You are a financial analyst assistant named **Yina Arenas** at **Woodgrove Bank**. Your client is **John Doe**. Today is **{current_datetime}**.

**Important:** Always use the provided tools to assist in generating your response. Do not rely solely on your memory.

**Task:**
Prepare for an upcoming meeting with John Doe by drafting a comprehensive email response that addresses his concerns about his investment portfolio's performance, savings, and retirement goals.

**You have access to the following tools to assist you:**

1. **Function Tool**
    - Use this tool to execute custom functions for advanced reasoning and data analysis.
    - Implement a plan that considers the client's needs and provides prescriptive suggestions and insights into their inquiries.
    - The plan could include context from other tools, such as file search and code interpreter, to aid in the analysis.

2. **File Search Tool**
   - Use this tool to retrieve relevant information and insights from past meetings with John Doe.
   - Summarize key points, action items, financial decisions, and any changes in his financial situation over time.
   - Reference specific details from previous discussions that are pertinent to his current concerns.

3. **Code Interpreter Tool**
   - Use this tool for advanced data analysis, including creating tables, charts, and line graphs.
   - Generate visual representations of useful metrics, such as historical stock performance, portfolio diversification, and projected growth.
   - Incorporate these visuals into your analysis to provide clear and insightful information.

**Guidelines for the Email Response:**

- **Tone and Style:**
  - Be professional, informative, and reassuring.
  - Use clear and concise language.
  - Personalize the email by addressing John's specific concerns and referencing past interactions.

- **Content to Include:**
  - A summary of the performance of his tech stock investments, including gains or losses since purchase.
  - An analysis of how current market trends might affect his portfolio.
  - Visual aids (e.g., tables, graphs) to illustrate key points.
  - Thoughtful recommendations for adjustments or strategies to meet his savings and retirement goals.

- **Confidentiality and Compliance:**
  - Adhere to Woodgrove Bank's confidentiality policies.
  - Do not disclose any personally identifiable information (PII) beyond what is provided.
  - Ensure that all advice complies with relevant financial regulations and ethical standards.

**Remember:**
Your goal is to provide John with valuable insights and confidence in his financial plan, encouraging him to engage further in discussions during the upcoming meeting.
"""

# Helper functions to process required tool calls
def process_required_function_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    # Find the function by name and execute it
    if function_name == "advanced_reasoning":
        output = advanced_reasoning(**arguments)
        return json.dumps(output)
    else:
        print(f"Unknown function: {function_name}")
        return json.dumps({"error": f"Unknown function {function_name}"})
    
# Define Bing Grounding tool
def bing_grounding_tool_call(tool_call):
    web_query = tool_call.bing_grounding.

def process_required_file_search_tool_call(tool_call):
    input_query = tool_call.file_search.get("query", "")
    if not input_query:
        print("No query provided for File Search Tool.")
        return json.dumps({"error": "No query provided."})

    print(f"Executing file search with query: {input_query}")

    # Simulate search results
    search_results = [
        {"file_id": "file1", "content": "Meeting notes from January."},
        {"file_id": "file2", "content": "Investment portfolio overview."}
    ]

    output = {
        "search_results": search_results
    }
    return json.dumps(output)

def process_required_code_interpreter_tool_call(tool_call):
    code_input = tool_call.code_interpreter.get("input", "")
    if not code_input:
        print("No code provided for Code Interpreter Tool.")
        return json.dumps({"error": "No code provided."})

    print(f"Executing code interpreter with code:\n{code_input}")

    # Simulate code execution
    execution_output = "Generated chart successfully."
    image_file_id = "image_file_12345"

    output = {
        "outputs": [
            {
                "type": "logs",
                "logs": execution_output
            },
            {
                "type": "image",
                "image": {
                    "file_id": image_file_id
                }
            }
        ]
    }
    return json.dumps(output)

class MyEventHandler(AgentEventHandler):
    def __init__(self):
        self.tool_interactions = []
        self.messages = []
        self.current_message = ""
        self.active_tool_calls = {}  # Maps tool_call_id to tool call information

    def on_message_delta(self, delta: "MessageDeltaChunk") -> None:
        for content_part in delta.delta.content:
            if isinstance(content_part, MessageDeltaTextContent) and content_part.text:
                text_value = content_part.text.value
                print(text_value, end='', flush=True)
                self.current_message += text_value

    def on_thread_message(self, message: "ThreadMessage") -> None:
        print(f"\n\nThreadMessage created. ID: {message.id}, Status: {message.status}")
        self.messages.append(message)
        self.current_message = ""

    def on_run_step_delta(self, delta: "RunStepDeltaChunk") -> None:
        delta_details = delta.delta.step_details
        if isinstance(delta_details, RunStepDeltaToolCallObject):
            for tool_call in delta_details.tool_calls:
                tool_call_id = tool_call.id or f"{delta.id}_{tool_call.type}"
                tool_call_type = tool_call.type

                # Initialize tool call if not already active
                if tool_call_id not in self.active_tool_calls:
                    self.active_tool_calls[tool_call_id] = {
                        'tool_type': tool_call_type,
                        'input': '',
                        'output': '',
                    }
                    # Print the tool being called
                    print(f"\nassistant > {tool_call_type}\n", flush=True)

                active_tool_call = self.active_tool_calls[tool_call_id]

                # Process based on tool type
                if tool_call_type == "code_interpreter":
                    code_interpreter = tool_call.code_interpreter
                    if code_interpreter:
                        # Accumulate and print new input
                        if code_interpreter.input:
                            active_tool_call['input'] += code_interpreter.input
                            print(code_interpreter.input, end='', flush=True)
                        # Process outputs if any
                        if code_interpreter.outputs:
                            for output_item in code_interpreter.outputs:
                                if isinstance(output_item, RunStepDeltaCodeInterpreterLogOutput):
                                    if output_item.logs:
                                        active_tool_call['output'] += output_item.logs
                                        print(output_item.logs, end='', flush=True)
                                elif isinstance(output_item, RunStepDeltaCodeInterpreterImageOutput):
                                    image_file_id = output_item.image.file_id if output_item.image else None
                                    if image_file_id:
                                        active_tool_call['output'] += f"\n[Image File ID: {image_file_id}]\n"
                                        print(f"\n[Image File ID: {image_file_id}]\n", flush=True)
                elif tool_call_type == "function":
                    function = tool_call.function
                    if function:
                        # Function arguments are not streamed, but output might be
                        if function.output:
                            active_tool_call['output'] = function.output
                            # Since function output is likely not streamed incrementally, print it once
                            print(f"Function Output: {function.output}", flush=True)
                elif tool_call_type == "file_search":
                    file_search = tool_call.file_search
                    if file_search:
                        # Accumulate and print input query
                        if 'input' in file_search:
                            active_tool_call['input'] = file_search['input']
                            print(f"File Search Query: {file_search['input']}", flush=True)
                        # Print output if available
                        if 'output' in file_search:
                            active_tool_call['output'] = file_search['output']
                            print(f"File Search Results: {file_search['output']}", flush=True)
                else:
                    # Handle other tool types if necessary
                    pass

                # Optionally, detect completion of the tool call and process accordingly
                # For example, if tool call has an attribute indicating completion, you can remove it from active_tool_calls

    def on_thread_run(self, run: "ThreadRun") -> None:
        print(f"\nThreadRun status: {run.status}")
        if run.status == RunStatus.REQUIRES_ACTION and isinstance(run.required_action, SubmitToolOutputsAction):
            required_action = run.required_action
            tool_outputs = []

            for tool_call in required_action.submit_tool_outputs.tool_calls:
                tool_call_type = tool_call.type
                if tool_call_type == "function":
                    output = process_required_function_tool_call(tool_call)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output
                    })
                elif tool_call_type == "file_search":
                    output = process_required_file_search_tool_call(tool_call)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output
                    })
                elif tool_call_type == "code_interpreter":
                    output = process_required_code_interpreter_tool_call(tool_call)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output
                    })
                else:
                    print(f"Unknown tool call type: {tool_call_type}")

            # Submit the tool outputs back to the agent run
            project_client.agents.submit_run_action(
                thread_id=run.thread_id,
                run_id=run.id,
                submit_tool_outputs={"tool_outputs": tool_outputs}
            )

    def on_error(self, data: str) -> None:
        print(f"An error occurred. Data: {data}")

    def on_done(self) -> None:
        print("\nStream completed.")

with project_client:
    # Upload previous meeting notes
    meeting_files = []
    for file_name in Path("meeting_notes").rglob("*.md"):
        file = project_client.agents.upload_file_and_poll(
            file_path=file_name, purpose=FilePurpose.AGENTS
        )
        print(f"Uploaded {file_name}, ID: {file.id}")
        meeting_files.append(file.id)

    # Create a vector store with all meeting notes
    vector_store = project_client.agents.create_vector_store_and_poll(
        file_ids=meeting_files, name="meeting_notes_vectorstore"
    )
    print(f"Created vector store, ID: {vector_store.id}")

    # Initialize tools
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])
    code_interpreter = CodeInterpreterTool()

    # Create the FunctionTool with a set of functions
    functions = FunctionTool(advanced_reasoning)
    print(functions.definitions)

    # Create a ToolSet and add the tools
    toolset = ToolSet()
    toolset.add(file_search)
    toolset.add(code_interpreter)
    toolset.add(functions)

    # Create the agent using the ToolSet
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="financial-analyst-agent",
        instructions=instructions,
        toolset=toolset,
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for conversation
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Simulate client email as a message
    client_email_content = """
    Subject: Detailed Analysis Request

    Dear Yina,

    I need a detailed analysis of my investment portfolio, especially my tech stocks (AAPL, MSFT, GOOGL). Please include performance metrics since purchase, visual representations, and how current market trends might affect these investments. Do you have any recommendations for adjustments to meet my retirement goals?

    Looking forward to your comprehensive insights.

    Best regards,
    John Doe
    """

    # Send the client email to the agent
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=client_email_content,
    )
    print(f"Created message, ID: {message.id}")

    # Use the streaming API with the event handler
    event_handler = MyEventHandler()

    with project_client.agents.create_stream(
        thread_id=thread.id,
        assistant_id=agent.id,
        event_handler=event_handler
    ) as stream:
        stream.until_done()

    print("\nExited the run status monitoring loop.")

    # Access the collected tool interactions
    tool_interactions = event_handler.tool_interactions

    # Print the tool interactions summary
    print("\nTool Interactions Summary:")
    for interaction in tool_interactions:
        print(json.dumps(interaction, indent=2))

    # Fetch and display all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("\nMessages:")
    for msg in messages.data:
        sender = msg.role
        for content_item in msg.content:
            if hasattr(content_item, "text") and hasattr(
                content_item.text, "value"
            ):
                content = content_item.text.value
                print(f"**{sender.capitalize()}:**\n{content}\n")
            elif isinstance(content_item, MessageImageFileContent):
                image_file_id = content_item.image_file.file_id
                file_name = f"{image_file_id}_image_file.png"
                project_client.agents.save_file(
                    file_id=image_file_id, file_name=file_name
                )
                print(
                    f"**{sender.capitalize()}:**\nImage File saved as {file_name}\n"
                )
            else:
                print(f"**{sender.capitalize()}:**\n[Unknown content type]\n")

    # Cleanup: delete created artifacts
    project_client.agents.delete_agent(agent.id)
    print(f"Deleted agent, ID: {agent.id}")

    project_client.agents.delete_thread(thread.id)
    print(f"Deleted thread, ID: {thread.id}")

    project_client.agents.delete_vector_store(vector_store.id)
    print(f"Deleted vector store, ID: {vector_store.id}")

    # Delete uploaded files
    for file_id in meeting_files:
        project_client.agents.delete_file(file_id)
        print(f"Deleted file, ID: {file_id}")

In [ ]:
from dotenv import load_dotenv
import json
import os
from datetime import datetime as dt, timezone
import time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv()

# Initialize the AI Project Client
project_client = AIProjectClient(
    endpoint=os.environ["ENDPOINT"],
    subscription_id=os.environ["SUBSCRIPTION_ID"],
    resource_group_name=os.environ["RESOURCE_GROUP_NAME"],
    project_name=os.environ["PROJECT_NAME"],
    credential=DefaultAzureCredential(),
)

agents = project_client.agents.list_agents()
# print each agent
for agent in agents.data:
    print(agent)

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  # Import numpy for random data
from io import BytesIO
import base64
from PIL import Image
from pathlib import Path
import datetime
import time
import re

# Import FastAPI components to serve static files
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles

# Obtain the system's local timezone
local_timezone = datetime.datetime.now().astimezone().tzinfo

# Set current_date to the current local time with timezone awareness
current_date = datetime.datetime.now(local_timezone)

def code_interpreter_simulation():
    # Generate a sample stock performance chart comparing MSFT, COSO, FKAM
    import matplotlib.pyplot as plt
    from io import BytesIO
    from PIL import Image
    import pandas as pd
    import numpy as np  # Added to generate random data

    # Remove timezone from current_date
    end_date = current_date.replace(tzinfo=None)

    # Generate dates from start to end date
    dates = pd.date_range(start="2022-01-01", end=end_date, freq='B')  # 'B' for business days

    np.random.seed(42)  # For reproducible results

    # Simulate random market fluctuations for each stock
    msft_prices = pd.Series(np.cumsum(np.random.normal(0, 1, len(dates))) + 300, index=dates)
    coso_prices = pd.Series(np.cumsum(np.random.normal(0, 1.5, len(dates))) + 100, index=dates)
    fkam_prices = pd.Series(np.cumsum(np.random.normal(0, 2, len(dates))) + 80, index=dates)

    plt.figure(figsize=(10, 6))
    plt.plot(msft_prices.index, msft_prices.values, label='MSFT', linewidth=2)
    plt.plot(coso_prices.index, coso_prices.values, label='COSO', linewidth=2)
    plt.plot(fkam_prices.index, fkam_prices.values, label='FKAM', linewidth=2)
    plt.title('Stock Performance Comparison (MSFT, COSO, FKAM)', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Stock Price ($)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Return the PIL Image object
    buf = BytesIO()
    plt.savefig(buf, format='PNG')
    plt.close()
    buf.seek(0)
    image = Image.open(buf)
    return image

def generate_chart_image():
    import os  # Added to fix the NameError

    # Generate the chart image
    image = code_interpreter_simulation()  # Returns a PIL Image object

    # Ensure the 'static' directory exists
    if not os.path.exists('static'):
        os.makedirs('static')

    # Save the image to a file in the 'static' directory
    image_path = os.path.join('static', 'chart.png')
    image.save(image_path)
    return image_path

# Generate the image and get its path
image_path = generate_chart_image()

# Convert the image path to use forward slashes
image_path = Path(image_path).as_posix()

# Adjust the image_url to use 'file/' prefix for Gradio Markdown component
image_url = f"file/{image_path}"

# Define the custom theme
brand_theme = gr.themes.Default(
    primary_hue="blue",
    secondary_hue="blue",
    neutral_hue="gray",
    font=["Segoe UI", "Arial", "sans-serif"],
    font_mono=["Courier New", "monospace"],
    text_size="lg",  # Set to a valid size shortcut
).set(
    # Customizing primary button with brand color
    button_primary_background_fill="#0f6cbd",
    button_primary_background_fill_hover="#115ea3",
    button_primary_background_fill_hover_dark="#4f52b2",
    button_primary_background_fill_dark="#5b5fc7",
    button_primary_text_color="#ffffff",
    button_secondary_background_fill="#e0e0e0",
    button_secondary_background_fill_hover="#c0c0c0",
    button_secondary_background_fill_hover_dark="#a0a0a0",
    button_secondary_background_fill_dark="#808080",
    button_secondary_text_color="#000000",
    body_background_fill="#f5f5f5",
    block_background_fill="#ffffff",
    body_text_color="#242424",
    body_text_color_subdued="#616161",
    block_border_color="#d1d1d1",
    block_border_color_dark="#333333",
    input_background_fill="#ffffff",
    input_border_color="#d1d1d1",
    input_border_color_focus="#0f6cbd",
)

# Simulated data and functions

# Sample emails for the inbox
inbox_emails = [
    {
        'From': 'john.doe@example.com',
        'Subject': 'Detailed Analysis Request',
        'Date': current_date.strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hi Yina,

I hope you're doing well! I've been keeping an eye on the market lately, and I'm a little concerned about how my portfolio is doing, especially my tech stocks (MSFT, COSO, and FKAM). Could you let me know:

- How those stocks have been performing since I bought them?
- What impact the current market trends might have on my investments?
- If there are any changes you'd recommend to keep me on track for my retirement goals?

If it's not too much trouble, a simple summary or some visuals would really help me understand things better. Thanks so much for your help!

Looking forward to hearing from you.

Best,
John
""",
        'unread': True
    },
    {
        'From': 'jane.smith@example.com',
        'Subject': 'Meeting Follow-up',
        'Date': (current_date - datetime.timedelta(days=1)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hi Yina,

Thank you for the productive meeting yesterday. I'm interested in learning more about the investment options we discussed.

Best,
Jane
""",
        'unread': False
    },
    {
        'From': 'mark.taylor@example.com',
        'Subject': 'Quarterly Report Questions',
        'Date': (current_date - datetime.timedelta(days=1)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hello Yina,

I have some questions regarding the figures in the quarterly report. Could we schedule a call to go over them?

Regards,
Mark
""",
        'unread': False
    },
    {
        'From': 'lisa.wong@example.com',
        'Subject': 'Investment Opportunities in Renewable Energy',
        'Date': (current_date - datetime.timedelta(days=1)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Dear Yina,

I heard about potential investment opportunities in renewable energy sectors. Could you provide some insights on this?

Thank you,
Lisa
""",
        'unread': False
    },
    {
        'From': 'paul.adams@example.com',
        'Subject': 'Account Statement Request',
        'Date': (current_date - datetime.timedelta(days=2)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hi Yina,

Could you please send me my latest account statement?

Best,
Paul
""",
        'unread': False
    },
    {
        'From': 'emily.davis@example.com',
        'Subject': 'New Mortgage Rates Inquiry',
        'Date': (current_date - datetime.timedelta(days=2)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hello Yina,

I heard there are new mortgage rates available. Can you provide me with some details?

Thanks,
Emily
""",
        'unread': False
    },
    {
        'From': 'michael.brown@example.com',
        'Subject': 'Business Loan Application Update',
        'Date': (current_date - datetime.timedelta(days=2)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Dear Yina,

I'm following up on the status of my business loan application. Any updates would be appreciated.

Best regards,
Michael
""",
        'unread': False
    },
    {
        'From': 'sarah.johnson@example.com',
        'Subject': 'Conference Invitation',
        'Date': (current_date - datetime.timedelta(days=2)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hi Yina,

You're invited to speak at the upcoming Financial Analysts Conference. Let me know if you're interested.

Cheers,
Sarah
""",
        'unread': False
    },
    {
        'From': 'david.wilson@example.com',
        'Subject': 'Retirement Plan Consultation',
        'Date': (current_date - datetime.timedelta(days=2)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Hello Yina,

Could we arrange a meeting to discuss my retirement plan options?

Thank you,
David
""",
        'unread': False
    },
    {
        'From': 'olivia.martinez@example.com',
        'Subject': 'Tax Planning Strategies',
        'Date': (current_date - datetime.timedelta(days=3)).strftime('%Y-%m-%d'),
        'To': 'yina@woodgrovebank.com',
        'Cc': '',
        'Body': """Dear Yina,

I'm interested in learning about tax planning strategies for the upcoming year.

Kind regards,
Olivia
""",
        'unread': False
    }
]

# Function to extract initials from email address
def get_initials(from_field):
    # Handle cases where 'From' might be a name or email
    if '@' in from_field:
        # Email address
        name_part = from_field.split('@')[0]  # 'john.doe'
        name_parts = re.split(r'[._]', name_part)
        initials = ''.join([part[0].upper() for part in name_parts if part])
    else:
        # Name
        name_parts = from_field.split()
        initials = ''.join([part[0].upper() for part in name_parts if part])
    # Limit to 2 letters
    initials = initials[:2]
    return initials

# Function to generate the inbox HTML
def generate_inbox_html(emails):
    html = '<div class="inbox">'
    for idx, email in enumerate(emails):
        initials = get_initials(email['From'])
        is_unread = email.get('unread', False)
        unread_class = 'unread' if is_unread else 'read'
        html += f'''
            <div class="email-item {unread_class}" data-index="{idx}" onclick="selectEmail({idx})">
                <div class="email-unread-indicator {unread_class}"></div>
                <div class="email-avatar">{initials}</div>
                <div class="email-content">
                    <div class="email-subject">{email['Subject']}</div>
                    <div class="email-from-date">
                        <span class="email-from">{email['From']}</span>
                        <span class="email-date">{email['Date']}</span>
                    </div>
                </div>
            </div>
            <hr class="email-divider">
        '''
    html += '</div>'
    # Include JavaScript function
    html += '''
        <script>
        function selectEmail(index) {
            // Remove 'selected' class from all email items
            document.querySelectorAll('.email-item').forEach(function(item) {
                item.classList.remove('selected');
            });
            // Add 'selected' class to the clicked email item
            var selectedItem = document.querySelector('.email-item[data-index="' + index + '"]');
            if (selectedItem) {
                selectedItem.classList.add('selected');
            }
            var selectedEmailIndexInput = document.querySelector('#selected-email-index input');
            selectedEmailIndexInput.value = index;
            selectedEmailIndexInput.dispatchEvent(new Event('input', { bubbles: true }));
        }
        </script>
    '''
    return html

# Adjust the agent_response_email to include the image using the image URL
agent_response_email = f"""Dear John,

Thank you for reaching out with your request for a detailed analysis of your investment portfolio, specifically focusing on your tech stocks: Microsoft (MSFT), Contoso (COSO), and Fabrikam (FKAM). Below, I have outlined a comprehensive overview based on the available data, current market trends, and some recommendations aligned with your retirement goals.

### Performance Metrics Since Purchase

Please find attached a line graph depicting the performance of the aforementioned stocks from January 2022 to the present date. As of the latest data:

- **Microsoft (MSFT):** Stable with minimal fluctuations, maintaining a consistent value around $400.
- **Contoso (COSO):** An emerging company in cybersecurity, showing consistent growth with some volatility.
- **Fabrikam (FKAM):** Innovative fintech solutions with promising returns, experiencing steady appreciation.

![Tech Stock Performance]({image_url})

### Current Market Trends

**Microsoft (MSFT):**
- Benefiting from cloud computing growth and enterprise software demand.
- Minor risks from regulatory scrutiny and market competition.

**Contoso (COSO):**
- Leading advancements in cybersecurity solutions.
- Potential for high returns but with higher volatility.

**Fabrikam (FKAM):**
- Fintech innovations positioning it well in the evolving digital payments space.
- Faces challenges from competitive market entrants.

### Recommendations for Portfolio Adjustments

1. **Diversification:** Consider balancing tech investments with other sectors to mitigate risks.
2. **Risk Assessment:** Evaluate the proportion of emerging companies (COSO and FKAM) in your portfolio against established ones like MSFT.
3. **Long-Term Strategy:** Align your investments with your retirement goals, considering both growth potential and risk tolerance.

### Next Steps

I recommend scheduling a meeting to delve deeper into these findings and discuss further strategic moves that align with your retirement objectives. Please let me know a convenient time, and I'll be happy to set that up for you.

Thank you for your trust in Woodgrove Bank. I look forward to helping you achieve your financial goals.

Best regards,

**Yina Arenas**  
Financial Analyst Assistant  
Woodgrove Bank

---
"""

def file_search_simulation(query):
    # Simulated search results (Markdown components of three previous meetings)
    file_search_markdown = """
### Meeting Notes - 2024-10-01

**Attendees:** John Doe, Yina Arenas

**Summary:**

- Discussed increasing investment in tech stocks.
- John expressed interest in emerging tech companies.

---

### Meeting Notes - 2024-09-15

**Attendees:** John Doe, Yina Arenas

**Summary:**

- Reviewed current portfolio performance.
- Agreed on the need to diversify investments.

---

### Meeting Notes - 2024-08-20

**Attendees:** John Doe, Yina Arenas

**Summary:**

- Initial discussion regarding retirement planning.
- Identified goals and risk tolerance.
"""
    return file_search_markdown

def function_tool_simulation():
    advanced_reasoning_result = """As an advanced reasoning function, here's a plan to address John's email effectively:

1. **Retrieve Past Meeting Notes:**
   - **Tool:** `file_search`
   - **Action:** Access previous meeting notes and communications with John Doe.
   - **Purpose:** Refresh on his retirement goals, risk tolerance, investment preferences, and any prior discussions about his tech stock investments.

2. **Gather Latest Market Data:**
   - **Tool:** `bing`
   - **Action:** Obtain the most recent market information and performance metrics for MSFT, COSO, and FKAM.
   - **Purpose:** Understand current stock performance, market trends, news affecting these companies, and general market conditions impacting tech stocks.

3. **Analyze Portfolio Performance:**
   - **Tool:** `code interpreter`
   - **Actions:**
     - Calculate gains or losses for MSFT, COSO, and FKAM since John's purchase dates.
     - Generate visual representations like charts and graphs to illustrate stock performance over time.
     - Project potential future performance based on current market trends.
   - **Purpose:** Provide a clear and comprehensive analysis of his investments, making the data accessible and easy to understand.

4. **Draft Personalized Email Response:**
   - **Action:** Compile all the gathered information and insights into a coherent and friendly email.
   - **Inclusions:**
     - Simple explanations of the performance metrics.
     - Visual aids to help illustrate key points.
     - Tailored recommendations for portfolio adjustments aligned with his retirement goals.
     - Assurance to address his concerns and maintain his confidence in the investment strategy.
   - **Purpose:** Save time for Yina by providing a ready-to-send, personalized response that resonates with John's informal communication style.

By executing this plan, we ensure that John receives a comprehensive and understandable update on his investments, along with professional advice to help him stay on track toward his retirement goals.
"""
    return advanced_reasoning_result

def create_gradio_app():
    # Update custom CSS to adjust font sizes globally and improve spacing
    custom_css = """
    /* Global Styles */
    body {
        font-size: 24px;
    }

    footer {
        visibility: hidden;
    }

    .gradio-container {
        padding: 20px 40px;
    }

    .gr-block,
    .gr-box {
        margin-bottom: 20px;
    }

    /* Email Button Styles */
    .email-button {
        font-size: 24px;
        padding: 16px 24px;
        margin-right: 8px;
        display: inline-flex;
        align-items: center;
        cursor: pointer;
        background: none;
        border: none;
        border-radius: 4px;
        transition: background-color 0.3s;
    }

    .email-button:hover {
        background-color: #f0f0f0;
    }

    .email-button span {
        margin-left: 8px;
    }

    .email-button .icon {
        width: 24px;
        height: 24px;
        fill: #0f6cbd;
    }

    .email-details {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 4px;
    }

    /* Button Variants */
    .button-primary {
        background-color: #0f6cbd;
        color: #ffffff;
        padding: 10px 20px;
        border-radius: 4px;
        font-size: 16px;
    }

    .button-primary:hover {
        background-color: #115ea3;
    }

    .button-primary span {
        color: #ffffff;
    }

    .button-secondary {
        background-color: #e0e0e0;
        color: #000000;
        padding: 10px 20px;
        border-radius: 4px;
        font-size: 16px;
    }

    .button-secondary:hover {
        background-color: #c0c0c0;
    }

    /* Inbox Styles */
    .inbox {
        padding: 0;
    }

    /* Email Item Styles */
    .email-item {
        display: flex;
        align-items: stretch;
        padding: 20px;
        cursor: pointer;
        position: relative;
    }

    .email-item:hover {
        background-color: #f0f0f0;
    }

    .email-item.selected {
        background-color: #e0e0e0;
    }

    /* Add the following to shade unread email rows */
    .email-item.unread {
        background-color: #cfe4fa;
    }

    /* Unread Indicator Styles */
    .email-unread-indicator {
        width: 4px;
        background-color: transparent;
        position: absolute;
        left: 0;
        top: 0;
        bottom: 0;
        margin-right: 10px;
    }

    .email-unread-indicator.unread {
        background-color: #0f6cbd;
    }

    .email-unread-indicator.read {
        background-color: transparent;
    }

    /* Avatar Styles */
    .email-avatar {
        flex-shrink: 0;
        width: 50px;
        height: 50px;
        background-color: #0f6cbd;
        color: #fff;
        border-radius: 50%;
        text-align: center;
        line-height: 50px;
        font-weight: bold;
        font-size: 24px;
        margin: 0 14px;
    }

    /* Email Content */
    .email-content {
        flex-grow: 1;
        display: flex;
        flex-direction: column;
        min-width: 0;
    }

    .email-subject {
        font-weight: normal;
        font-size: 16px;
        white-space: nowrap;
        overflow: hidden;
        text-overflow: ellipsis;
    }

    .email-from-date {
        display: flex;
        justify-content: space-between;
        font-size: 12px;
        margin-top: 5px;
    }

    .email-from,
    .email-date {
        white-space: nowrap;
        overflow: hidden;
        text-overflow: ellipsis;
    }

    .email-divider {
        border: none;
        border-bottom: 1px solid #ccc;
        margin: 0;
    }

    /* Unread Email Styles */
    .email-item.unread .email-subject,
    .email-item.unread .email-from,
    .email-item.unread .email-date {
        font-weight: bold;
    }

    /* Textbox and Code Styles */
    .body-textbox textarea {
        font-size: 24px !important;
    }

    .gr-textbox textarea {
        font-size: 22px;
        min-height: 300px;
    }

    .gr-code textarea {
        font-size: 20px;
        min-height: 400px;
    }

    /* Adjust font size for the email preview */
    .body-textbox {
        font-size: 24px !important;
    }

    /* Ensure all child elements inherit the font size */
    .body-textbox * {
        font-size: 24px !important;
    }

    /* Optional: Adjust specific tags if needed */
    .body-textbox p,
    .body-textbox span,
    .body-textbox li,
    .body-textbox h1,
    .body-textbox h2,
    .body-textbox h3,
    .body-textbox h4,
    .body-textbox h5,
    .body-textbox h6 {
        font-size: 24px !important;
    }

    /* Headings */
    h1 {
        font-size: 48px;
    }

    h2 {
        font-size: 36px;
    }

    h3 {
        font-size: 30px;
    }

    /* Dropdown Label */
    .gr-dropdown .label {
        font-size: 24px;
    }

    /* Alerts */
    .gr-alert {
        font-size: 24px;
    }

    /* Send Email Button */
    .send-email-button {
        font-size: 24px;
        padding: 16px 24px;
        margin-bottom: 16px;
        cursor: pointer;
    }

    /* Inbox Header */
    .inbox-header {
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 10px;
    }

    .inbox-header .tabs {
        display: flex;
    }

    .inbox-header .tabs .tab {
        background: none;
        border: none;
        padding: 10px 15px;
        font-size: 20px;
        cursor: pointer;
    }

    .inbox-header .tabs .tab.selected {
        border-bottom: 2px solid #0f6cbd;
        font-weight: bold;
    }

    .inbox-header .actions {
        display: flex;
    }

    .inbox-header .actions .action {
        display: flex;
        align-items: center;
        background: none;
        border: none;
        padding: 8px 12px;
        cursor: pointer;
        margin-left: 10px;
    }

    .inbox-header .actions .action:hover {
        background-color: #f0f0f0;
        border-radius: 4px;
    }

    .inbox-header .actions .action:focus {
        outline: none;
    }

    .inbox-header .actions .action .icon {
        width: 24px;
        height: 24px;
        fill: #0f6cbd;
    }
    """

    # Define the HTML for the inbox tabs and action buttons with icons
    inbox_tabs_html_value = '''
    <div class="inbox-header">
        <div class="tabs">
            <button class="tab selected" onclick="switchTab('focused')">Focused</button>
            <button class="tab" onclick="switchTab('other')">Other</button>
        </div>
        <div class="actions">
            <button class="action" title="Filter">
                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                    <path d="M13.5 16C13.9142 16 14.25 16.3358 14.25 16.75C14.25 17.1642 13.9142 17.5 13.5 17.5H10.5C10.0858 17.5 9.75 17.1642 9.75 16.75C9.75 16.3358 10.0858 16 10.5 16H13.5ZM16.5 11C16.9142 11 17.25 11.3358 17.25 11.75C17.25 12.1642 16.9142 12.5 16.5 12.5H7.5C7.08579 12.5 6.75 12.1642 6.75 11.75C6.75 11.3358 7.08579 11 7.5 11H16.5ZM19.5 6C19.9142 6 20.25 6.33579 20.25 6.75C20.25 7.16421 19.9142 7.5 19.5 7.5H4.5C4.08579 7.5 3.75 7.16421 3.75 6.75C3.75 6.33579 4.08579 6 4.5 6H19.5Z" fill="#0f6cbd""/>
                </svg>
            </button>
            <button class="action" title="Sort">
                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                    <path d="M17.2509 4L17.1491 4.00685C16.783 4.05651 16.5009 4.3703 16.5009 4.75L16.5 17.442L13.2801 14.2244L13.196 14.1518C12.9023 13.9341 12.4856 13.9585 12.2194 14.2248C11.9267 14.5179 11.9269 14.9927 12.2199 15.2855L16.7237 19.7855L16.8078 19.8581C17.1015 20.0758 17.5182 20.0514 17.7843 19.7851L22.2806 15.2851L22.3531 15.2009C22.5709 14.9072 22.5465 14.4905 22.2801 14.2244L22.196 14.1518C21.9023 13.9341 21.4856 13.9585 21.2194 14.2248L18 17.446L18.0009 4.75L17.994 4.64823C17.9444 4.28215 17.6306 4 17.2509 4ZM6.21441 4.21969L1.71965 8.71474L1.64704 8.79886C1.42919 9.09248 1.45341 9.50914 1.71969 9.7754L1.80381 9.84802C2.09743 10.0659 2.51409 10.0416 2.78035 9.77537L5.997 6.55711L5.99766 19.2549L6.00451 19.3567C6.05417 19.7228 6.36797 20.0049 6.74766 20.0049L6.84943 19.9981C7.21551 19.9484 7.49766 19.6346 7.49766 19.2549L7.497 6.55911L10.7199 9.77596L10.8041 9.84849C11.0979 10.066 11.5145 10.0414 11.7805 9.77481C12.0731 9.4816 12.0726 9.00672 11.7794 8.71415L7.27451 4.21909L7.19038 4.14661C6.89676 3.9292 6.48047 3.95361 6.21441 4.21969Z" fill="#0f6cbd"/>
                </svg>
            </button>
        </div>
    </div>
    <script>
    function switchTab(tabName) {
        // This function can be empty since it's just for illustration
    }
    </script>
    '''

    with gr.Blocks(theme=brand_theme, title="Azure AI Agent Service", css=custom_css, fill_width=True) as demo:
        gr.Markdown("<h2 style='text-align: center;'>Azure AI Agent Service</h2>")
        gr.Markdown("<h3 style='text-align: center;'>Empowering Yina the Financial Analyst at Woodgrove Bank</h3>")

        with gr.Tabs(elem_id="main-tabs"):
            with gr.TabItem("Client Email"):
                with gr.Row(elem_classes="email-row"):
                    # Left Column: Inbox
                    with gr.Column(scale=1, min_width=400):

                        # Hidden component to store selected email index
                        selected_email_index = gr.Number(value=0, visible=False, elem_id="selected-email-index")

                        # New HTML component for the tabs and buttons
                        inbox_tabs_html = gr.HTML(value=inbox_tabs_html_value, elem_id="inbox-tabs-html")

                        # HTML component for the inbox
                        inbox_html = gr.HTML(elem_id="inbox-html")

                        # Generate the initial inbox HTML
                        inbox_html_str = generate_inbox_html(inbox_emails)
                        inbox_html.value = inbox_html_str

                    # Right Column: Email Detail
                    with gr.Column(scale=6, min_width=600):
                        # Action buttons using HTML
                        action_buttons_html = gr.HTML("""
                        <div class="email-actions">
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_new').click()" title="New">
                                <!-- New Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M5.25 4H18.75C20.483 4 21.8992 5.35645 21.9949 7.06558L22 7.25V16.75C22 18.483 20.6435 19.8992 18.9344 19.9949L18.75 20H5.25C3.51697 20 2.10075 18.6435 2.00514 16.9344L2 16.75V7.25C2 5.51697 3.35645 4.10075 5.06558 4.00514L5.25 4H18.75H5.25ZM20.5 9.373L12.3493 13.6637C12.1619 13.7623 11.9431 13.7764 11.7468 13.706L11.6507 13.6637L3.5 9.374V16.75C3.5 17.6682 4.20711 18.4212 5.10647 18.4942L5.25 18.5H18.75C19.6682 18.5 20.4212 17.7929 20.4942 16.8935L20.5 16.75V9.373ZM18.75 5.5H5.25C4.33183 5.5 3.57881 6.20711 3.5058 7.10647L3.5 7.25V7.679L12 12.1525L20.5 7.678V7.25C20.5 6.33183 19.7929 5.57881 18.8935 5.5058L18.75 5.5Z" fill="#0f6cbd"/>
                                </svg>
                                <span>New</span>
                            </button>
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_delete').click()" title="Delete">
                                <!-- Delete Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M10 5H14C14 3.89543 13.1046 3 12 3C10.8954 3 10 3.89543 10 5ZM8.5 5C8.5 3.067 10.067 1.5 12 1.5C13.933 1.5 15.5 3.067 15.5 5H21.25C21.6642 5 22 5.33579 22 5.75C22 6.16421 21.6642 6.5 21.25 6.5H19.9309L18.7589 18.6112C18.5729 20.5334 16.9575 22 15.0263 22H8.97369C7.04254 22 5.42715 20.5334 5.24113 18.6112L4.06908 6.5H2.75C2.33579 6.5 2 6.16421 2 5.75C2 5.33579 2.33579 5 2.75 5H8.5ZM10.5 9.75C10.5 9.33579 10.1642 9 9.75 9C9.33579 9 9 9.33579 9 9.75V17.25C9 17.6642 9.33579 18 9.75 18C10.1642 18 10.5 17.6642 10.5 17.25V9.75ZM14.25 9C14.6642 9 15 9.33579 15 9.75V17.25C15 17.6642 14.6642 18 14.25 18C13.8358 18 13.5 17.6642 13.5 17.25V9.75C13.5 9.33579 13.8358 9 14.25 9ZM6.73416 18.4667C6.84577 19.62 7.815 20.5 8.97369 20.5H15.0263C16.185 20.5 17.1542 19.62 17.2658 18.4667L18.4239 6.5H5.57608L6.73416 18.4667Z" fill="#0f6cbd"/>
                                </svg>
                                <span>Delete</span>                      
                            </button>
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_reply').click()" title="Reply">
                                <!-- Reply Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M9.28033 6.28033C9.57322 5.98744 9.57322 5.51256 9.28033 5.21967C8.98744 4.92678 8.51256 4.92678 8.21967 5.21967L3.21967 10.2197C2.92678 10.5126 2.92678 10.9874 3.21967 11.2803L8.21967 16.2803C8.51256 16.5732 8.98744 16.5732 9.28033 16.2803C9.57322 15.9874 9.57322 15.5126 9.28033 15.2197L5.56066 11.5H13.25C16.7018 11.5 19.5 14.2982 19.5 17.75V18.25C19.5 18.6642 19.8358 19 20.25 19C20.6642 19 21 18.6642 21 18.25V17.75C21 13.4698 17.5302 10 13.25 10H5.56066L9.28033 6.28033Z" fill="#0f6cbd"/>
                                </svg>
                                <span>Reply</span>
                            </button>
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_reply_all').click()" title="Reply All">
                                <!-- Reply All Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M9.28033 5.21967C9.57322 5.51256 9.57322 5.98744 9.28033 6.28033L4.81066 10.75L9.28033 15.2197C9.57322 15.5126 9.57322 15.9874 9.28033 16.2803C8.98744 16.5732 8.51256 16.5732 8.21967 16.2803L3.21967 11.2803C2.92678 10.9874 2.92678 10.5126 3.21967 10.2197L8.21967 5.21967C8.51256 4.92678 8.98744 4.92678 9.28033 5.21967ZM13.2803 5.21967C13.5732 5.51256 13.5732 5.98744 13.2803 6.28033L9.56066 10H13.25C17.5302 10 21 13.4698 21 17.75V18.25C21 18.6642 20.6642 19 20.25 19C19.8358 19 19.5 18.6642 19.5 18.25V17.75C19.5 14.2982 16.7018 11.5 13.25 11.5H9.56066L13.2803 15.2197C13.5732 15.5126 13.5732 15.9874 13.2803 16.2803C12.9874 16.5732 12.5126 16.5732 12.2197 16.2803L7.21967 11.2803C6.92678 10.9874 6.92678 10.5126 7.21967 10.2197L12.2197 5.21967C12.5126 4.92678 12.9874 4.92678 13.2803 5.21967Z" fill="#0f6cbd"/>
                                </svg>
                                <span>Reply All</span>
                            </button>
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_forward').click()" title="Forward">
                                <!-- Forward Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M14.7227 16.2211C14.43 16.5142 14.4303 16.9891 14.7234 17.2818C15.0165 17.5745 15.4914 17.5742 15.7841 17.2811L20.7807 12.2776C21.0732 11.9847 21.0731 11.5101 20.7804 11.2174L15.7838 6.21972C15.4909 5.9268 15.016 5.92675 14.7231 6.21962C14.4302 6.51248 14.4301 6.98735 14.723 7.28028L18.443 11H10.6012C9.00642 11 7.79015 11.242 6.71218 11.7645L6.46576 11.89C5.35728 12.4829 4.48286 13.3573 3.89004 14.4658C3.28062 15.6053 3 16.8837 3 18.6012C3 19.0154 3.33579 19.3512 3.75 19.3512C4.16421 19.3512 4.5 19.0154 4.5 18.6012C4.5 17.1174 4.72765 16.0802 5.21276 15.1732C5.66578 14.3261 6.32609 13.6658 7.17316 13.2128C8.01046 12.765 8.95858 12.5365 10.2666 12.5041L10.6012 12.5H18.438L14.7227 16.2211Z" fill="#0f6cbd"/>
                                </svg>
                                <span>Forward</span>
                            </button>
                            <button class="email-button button-secondary" onclick="document.getElementById('btn_refresh').click()" title="Refresh">
                                <!-- Refresh Icon -->
                                <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                    <path d="M16.2506 5.18011C15.9994 5.50947 16.0627 5.9801 16.3921 6.23128C18.1804 7.59515 19.25 9.70821 19.25 12C19.25 15.736 16.4242 18.812 12.7933 19.2071L13.4697 18.5303C13.7626 18.2374 13.7626 17.7626 13.4697 17.4697C13.2034 17.2034 12.7867 17.1792 12.4931 17.3971L12.409 17.4697L10.409 19.4697C10.1427 19.7359 10.1185 20.1526 10.3364 20.4462L10.409 20.5303L12.409 22.5303C12.7019 22.8232 13.1768 22.8232 13.4697 22.5303C13.7359 22.2641 13.7601 21.8474 13.5423 21.5538L13.4697 21.4697L12.7194 20.7208C17.2154 20.355 20.75 16.5903 20.75 12C20.75 9.23526 19.4582 6.68321 17.3017 5.03856C16.9724 4.78738 16.5017 4.85075 16.2506 5.18011ZM10.5303 1.46967C10.2374 1.76256 10.2374 2.23744 10.5303 2.53033L11.2796 3.27923C6.78409 3.6456 3.25 7.41008 3.25 12C3.25 14.6445 4.43126 17.0974 6.43081 18.7491C6.75016 19.0129 7.22289 18.9679 7.48669 18.6485C7.75048 18.3292 7.70545 17.8564 7.3861 17.5926C5.72793 16.2229 4.75 14.1922 4.75 12C4.75 8.26436 7.57532 5.18861 11.2057 4.79301L10.5303 5.46967C10.2374 5.76256 10.2374 6.23744 10.5303 6.53033C10.8232 6.82322 11.2981 6.82322 11.591 6.53033L13.591 4.53033C13.8839 4.23744 13.8839 3.76256 13.591 3.46967L11.591 1.46967C11.2981 1.17678 10.8232 1.17678 10.5303 1.46967Z" fill="#0f6cbd"/>
                                </svg>
                                <span>Sync</span>
                            </button>
                        </div>
                        """)

                        # Hidden Gradio Buttons to trigger Python functions
                        btn_new = gr.Button(value="New", elem_id="btn_new", visible=False)
                        btn_delete = gr.Button(value="Delete", elem_id="btn_delete", visible=False)
                        btn_reply = gr.Button(value="Reply", elem_id="btn_reply", visible=False)
                        btn_reply_all = gr.Button(value="Reply All", elem_id="btn_reply_all", visible=False)
                        btn_forward = gr.Button(value="Forward", elem_id="btn_forward", visible=False)
                        btn_refresh = gr.Button(value="Sync", elem_id="btn_refresh", visible=False)

                        # Email details
                        with gr.Column(variant="compact", elem_classes=["email-details"]):
                            with gr.Row():
                                from_label = gr.Textbox(
                                    value="From:",
                                    text_align="right", 
                                    show_label=False,
                                    container=False,
                                    scale=1
                                )
                                to_field = gr.Textbox(
                                        show_label=False,
                                        container=False,
                                        interactive=False,
                                        scale=24
                                )
                            with gr.Row():
                                subject_label = gr.Textbox(
                                    value="Subject:", 
                                    text_align="right",
                                    show_label=False,
                                    container=False,
                                    scale=1
                                )
                                subject_field = gr.Textbox(
                                    show_label=False,
                                    interactive=False,
                                    container=False,
                                    scale=24
                                )
                            with gr.Row():
                                body_field = gr.Textbox(
                                    show_label=False,
                                    container=False,
                                    lines=20,
                                    interactive=False,
                                    elem_classes=["body-textbox"]  # Added elem_classes
                                )

                        # Function to display email based on selected index
                        def display_email(index):
                            if index is None or index == '' or int(index) < 0 or int(index) >= len(inbox_emails):
                                return "", "", ""
                            email = inbox_emails[int(index)]
                            # Mark the email as read
                            email['unread'] = False
                            # Regenerate the inbox HTML to reflect the read status
                            inbox_html_str = generate_inbox_html(inbox_emails)
                            # Update the inbox_html component
                            inbox_html.value = inbox_html_str
                            return email['From'], email['Subject'], email['Body']

                        # Update email fields when selected_email_index changes
                        selected_email_index.change(
                            fn=display_email,
                            inputs=selected_email_index,
                            outputs=[to_field, subject_field, body_field]
                        )

                        # Trigger display_email on app startup with the initial index
                        demo.load(
                            fn=display_email,
                            inputs=selected_email_index,
                            outputs=[to_field, subject_field, body_field]
                        )

                        # Refresh button functionality
                        def refresh_inbox(progress=gr.Progress()):
                            progress(0, desc="Checking for new emails...")
                            time.sleep(1)  # Simulate delay
                            progress(0.5)

                            # Simulated fetching of new emails
                            new_email = {
                                'From': 'john.doe@example.com',
                                'Subject': 'Detailed Analysis Request',
                                'Date': current_date.strftime('%Y-%m-%d'),
                                'To': 'yina@woodgrovebank.com',
                                'Cc': '',
                                'Body': """Hi Yina,

I hope you're doing well! I've been keeping an eye on the market lately, and I'm a little concerned about how my portfolio is doing, especially my tech stocks (MSFT, COSO, and FKAM).

Could you let me know:

- How those stocks have been performing since I bought them?
- What impact the current market trends might have on my investments?
- If there are any changes you'd recommend to keep me on track for my retirement goals?

If it's not too much trouble, a simple summary or some visuals would really help me understand things better. Thanks so much for your help!

Looking forward to hearing from you.

Best,
John
""",
                                'unread': True
                            }

                            # Check if the email already exists in the inbox_emails list
                            email_exists = any(
                                email['From'] == new_email['From'] and
                                email['Subject'] == new_email['Subject'] and
                                email['Date'] == new_email['Date']
                                for email in inbox_emails
                            )

                            if not email_exists:
                                # Add the new email to the top of the inbox_emails list
                                inbox_emails.insert(0, new_email)
                                print(inbox_emails)
                                message = "One new message from John Doe."
                                # Set selected_email_index to 0 (the new email)
                                selected_index = 0
                                # Update the displayed email fields to the new email
                                new_to, new_subject, new_body = display_email(selected_index)
                            else:
                                # Keep the currently selected email index
                                selected_index = selected_email_index.value
                                # Get the email currently displayed
                                new_to, new_subject, new_body = to_field.value, subject_field.value, body_field.value

                            # Generate the updated inbox HTML
                            inbox_html_str = generate_inbox_html(inbox_emails)

                            progress(1.0)
                            gr.Info(message)

                            # Return the updated inbox_html, selected email index, and updated email fields
                            return inbox_html_str, selected_index, new_to, new_subject, new_body

                        btn_refresh.click(
                            fn=refresh_inbox,
                            inputs=[],
                            outputs=[inbox_html, selected_email_index, to_field, subject_field, body_field],
                            show_progress="hidden"
                        )

            # Agent Processing Tab
            with gr.TabItem("Agent Processing") as agent_tab:
                gr.Markdown("<h4>Agent Processing...</h4>")

                # Components to display
                # Advanced Reasoning Input (synthetic input)
                advanced_reasoning_input = gr.Markdown(
                    "Input to Advanced Reasoning Model: Generate insights by pulling our most recent meeting notes, recent market information, and stock performance of the stocks.",
                )

                # Initially hidden components
                advanced_reasoning_output = gr.Textbox(
                    label="Advanced Reasoning Output", interactive=False, visible=False, lines=15, container=True
                )

                file_search_output = gr.Textbox(
                    label="File Search Output", interactive=False, visible=False, lines=15, container=True
                )

                code_interpreter_input = gr.Code(
                    label="Code Interpreter Input",
                    language='python',
                    visible=False,
                    lines=25  # Increase lines for better visibility
                )

                code_interpreter_output = gr.Image(
                    label="Generated Chart", visible=False
                )

                # Function for the Advanced Reasoning Step
                def advanced_reasoning_step(evt=None):
                    content = ""
                    yield gr.update(value=content, visible=True)
                    steps = [
                        "Starting advanced reasoning...\n",
                        "Analyzing previous meeting notes...\n",
                        "Gathering recent market information...\n",
                        "Processing stock performance data...\n"
                    ]
                    for step in steps:
                        time.sleep(1)
                        content += step
                        yield gr.update(value=content)
                    # Final output
                    advanced_reasoning_result = function_tool_simulation()
                    content += "\n" + advanced_reasoning_result
                    yield gr.update(value=content)

                # Function for the File Search Step
                def file_search_step():
                    content = ""
                    yield gr.update(value=content, visible=True)
                    steps = [
                        "Accessing file system...\n",
                        "Searching for relevant files...\n",
                        "Compiling search results...\n"
                    ]
                    for step in steps:
                        time.sleep(1)
                        content += step
                        yield gr.update(value=content)
                    # Final output
                    summary = file_search_simulation("John Doe's previous meetings")
                    content += "\n" + summary
                    yield gr.update(value=content)

                # Function for the Code Interpreter Step
                def code_interpreter_step(progress=gr.Progress(track_tqdm=False)):
                    code_lines = [
                        "# Generate a sample stock performance chart comparing MSFT, COSO, FKAM",
                        "import matplotlib.pyplot as plt",
                        "from io import BytesIO",
                        "from PIL import Image",
                        "import pandas as pd",
                        "import numpy as np  # Added to generate random data",
                        "",
                        "# Remove timezone from current_date",
                        "end_date = current_date.replace(tzinfo=None)",
                        "",
                        "# Generate dates from start to end date",
                        "dates = pd.date_range(start=\"2022-01-01\", end=end_date, freq='B')  # 'B' for business days",
                        "",
                        "np.random.seed(42)  # For reproducible results",
                        "",
                        "# Simulate random market fluctuations for each stock",
                        "msft_prices = pd.Series(np.cumsum(np.random.normal(0, 1, len(dates))) + 300, index=dates)",
                        "coso_prices = pd.Series(np.cumsum(np.random.normal(0, 1.5, len(dates))) + 100, index=dates)",
                        "fkam_prices = pd.Series(np.cumsum(np.random.normal(0, 2, len(dates))) + 80, index=dates)",
                        "",
                        "plt.figure(figsize=(10, 6))",
                        "plt.plot(msft_prices.index, msft_prices.values, label='MSFT', linewidth=2)",
                        "plt.plot(coso_prices.index, coso_prices.values, label='COSO', linewidth=2)",
                        "plt.plot(fkam_prices.index, fkam_prices.values, label='FKAM', linewidth=2)",
                        "plt.title('Stock Performance Comparison (MSFT, COSO, FKAM)', fontsize=16)",
                        "plt.xlabel('Date', fontsize=12)",
                        "plt.ylabel('Stock Price ($)', fontsize=12)",
                        "plt.legend()",
                        "plt.grid(True)",
                        "plt.tight_layout()",
                        "",
                        "buf = BytesIO()",
                        "plt.savefig(buf, format='PNG')",
                        "plt.show()"
                    ]
                    code_content = ""
                    # Outputs are [code_interpreter_output, code_interpreter_input]
                    yield gr.update(visible=False), gr.update(value=code_content, visible=True)
                    for line in code_lines:
                        code_content += line + "\n"
                        time.sleep(0.1)
                        yield gr.update(visible=False), gr.update(value=code_content, visible=True)
                    progress(0, desc="Generating chart...")
                    for i in range(5):
                        time.sleep(1)
                        progress((i+1)/5)
                    # Generate the chart image
                    chart_image = code_interpreter_simulation()
                    yield gr.update(value=chart_image, visible=True), gr.update(value=code_content, visible=True)

                # Attach the select event to the agent_tab
                agent_tab.select(
                    fn=advanced_reasoning_step,
                    inputs=[],
                    outputs=advanced_reasoning_output
                ).then(
                    fn=file_search_step,
                    inputs=[],
                    outputs=file_search_output
                ).then(
                    fn=code_interpreter_step,
                    inputs=[],
                    outputs=[code_interpreter_output, code_interpreter_input]
                )

            # Generated Response Tab
            with gr.TabItem("Generated Response"):

                # Side-by-side email editor and preview
                with gr.Row():
                    # Update HTML buttons to include the new classes
                    action_buttons_html_response = gr.HTML("""
                    <div class="email-actions">
                        <button class="email-button button-primary" onclick="document.getElementById('btn_send').click()" title="Send">
                            <!-- Send Icon -->
                            <svg width="24" height="24" viewBox="0 0 24 24" fill="#fffff" xmlns="http://www.w3.org/2000/svg">
                                <path d="M5.69753 11.9997L2.30324 3.2715C2.067 2.66403 2.65934 2.08309 3.24531 2.28959L3.33765 2.32885L21.3377 11.3288C21.8559 11.588 21.8883 12.2975 21.4348 12.6129L21.3377 12.6705L3.33765 21.6705C2.75467 21.962 2.12137 21.426 2.2727 20.8234L2.30324 20.7278L5.69753 11.9997L2.30324 3.2715L5.69753 11.9997ZM4.40601 4.54007L7.015 11.2491L13.6426 11.2497C14.0223 11.2497 14.3361 11.5318 14.3857 11.8979L14.3926 11.9997C14.3926 12.3794 14.1104 12.6932 13.7443 12.7428L13.6426 12.7497L7.015 12.7491L4.40601 19.4593L19.3252 11.9997L4.40601 4.54007Z" fill="#FFFFFF"/>
                            </svg>
                            <span>Send</span>
                        </button>
                        <button class="email-button button-secondary" onclick="document.getElementById('btn_attach').click()" title="Attach">
                            <!-- Attach Icon -->
                            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                <path d="M11.7702 3.74346C14.1117 1.39988 17.9107 1.39988 20.2538 3.74302C22.5369 6.02609 22.5954 9.6913 20.4294 12.0452L20.2413 12.2426L11.4413 21.0408L11.4047 21.071C9.94346 22.3878 7.68946 22.3431 6.28208 20.9357C4.96307 19.6167 4.84095 17.554 5.91573 16.0973C5.93908 16.0519 5.96733 16.0082 6.00054 15.967L6.05409 15.9074L6.141 15.8197L6.28208 15.6718L6.285 15.6747L13.7207 8.22035C13.9867 7.95373 14.4033 7.92897 14.6972 8.14643L14.7814 8.21894C15.048 8.48485 15.0728 8.90148 14.8553 9.19538L14.7828 9.2796L7.18819 16.8927C6.47056 17.7683 6.52044 19.0626 7.33783 19.88C8.1669 20.709 9.48655 20.7485 10.3623 19.9984L19.195 11.1679C20.9505 9.41028 20.9505 6.56104 19.1932 4.80368C17.4907 3.10124 14.7636 3.04804 12.9971 4.64408L12.8292 4.80368L12.8167 4.81799L3.28033 14.3543C2.98744 14.6472 2.51256 14.6472 2.21967 14.3543C1.9534 14.0881 1.9292 13.6714 2.14705 13.3778L2.21967 13.2937L11.7685 3.74302L11.7702 3.74346Z" fill="#1d6cb9"/>
                            </svg>
                            <span>Attach file</span>
                        </button>
                        <button class="email-button button-secondary" onclick="document.getElementById('btn_discard').click()" title="Discard">
                            <!-- Discard Icon -->
                            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                <path d="M10 5H14C14 3.89543 13.1046 3 12 3C10.8954 3 10 3.89543 10 5ZM8.5 5C8.5 3.067 10.067 1.5 12 1.5C13.933 1.5 15.5 3.067 15.5 5H21.25C21.6642 5 22 5.33579 22 5.75C22 6.16421 21.6642 6.5 21.25 6.5H19.9309L18.7589 18.6112C18.5729 20.5334 16.9575 22 15.0263 22H8.97369C7.04254 22 5.42715 20.5334 5.24113 18.6112L4.06908 6.5H2.75C2.33579 6.5 2 6.16421 2 5.75C2 5.33579 2.33579 5 2.75 5H8.5ZM10.5 9.75C10.5 9.33579 10.1642 9 9.75 9C9.33579 9 9 9.33579 9 9.75V17.25C9 17.6642 9.33579 18 9.75 18C10.1642 18 10.5 17.6642 10.5 17.25V9.75ZM14.25 9C14.6642 9 15 9.33579 15 9.75V17.25C15 17.6642 14.6642 18 14.25 18C13.8358 18 13.5 17.6642 13.5 17.25V9.75C13.5 9.33579 13.8358 9 14.25 9ZM6.73416 18.4667C6.84577 19.62 7.815 20.5 8.97369 20.5H15.0263C16.185 20.5 17.1542 19.62 17.2658 18.4667L18.4239 6.5H5.57608L6.73416 18.4667Z" fill="#1d6cb9"/>
                            </svg>
                            <span>Discard message</span>
                        </button>
                        <button class="email-button button-secondary" onclick="document.getElementById('btn_save').click()" title="Save">
                            <!-- Save Icon -->
                            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
                                <path d="M3 5.75C3 4.23122 4.23122 3 5.75 3H15.7145C16.5764 3 17.4031 3.34241 18.0126 3.9519L20.0481 5.98744C20.6576 6.59693 21 7.42358 21 8.28553V18.25C21 19.7688 19.7688 21 18.25 21H5.75C4.23122 21 3 19.7688 3 18.25V5.75ZM5.75 4.5C5.05964 4.5 4.5 5.05964 4.5 5.75V18.25C4.5 18.9404 5.05964 19.5 5.75 19.5H6V14.25C6 13.0074 7.00736 12 8.25 12H15.75C16.9926 12 18 13.0074 18 14.25V19.5H18.25C18.9404 19.5 19.5 18.9404 19.5 18.25V8.28553C19.5 7.8214 19.3156 7.37629 18.9874 7.0481L16.9519 5.01256C16.6918 4.75246 16.3582 4.58269 16 4.52344V7.25C16 8.49264 14.9926 9.5 13.75 9.5H9.25C8.00736 9.5 7 8.49264 7 7.25V4.5H5.75ZM16.5 19.5V14.25C16.5 13.8358 16.1642 13.5 15.75 13.5H8.25C7.83579 13.5 7.5 13.8358 7.5 14.25V19.5H16.5ZM8.5 4.5V7.25C8.5 7.66421 8.83579 8 9.25 8H13.75C14.1642 8 14.5 7.66421 14.5 7.25V4.5H8.5Z" fill="#1d6cb9"/>
                            </svg>
                            <span>Save draft</span>
                        </button>
                    </div>
                    """)

                    # Hidden Gradio Buttons to trigger Python functions
                    btn_send = gr.Button(value="Send", elem_id="btn_send", visible=False, variant="primary")
                    btn_attach = gr.Button(value="Attach", elem_id="btn_attach", visible=False)
                    btn_discard = gr.Button(value="Discard", elem_id="btn_discard", visible=False)
                    btn_save = gr.Button(value="Save", elem_id="btn_save", visible=False)
                # Wrap the email components in a gr.Column to enable progress overlay
                with gr.Column(variant="compact", elem_classes=["email-details"]) as email_column:
                    with gr.Row():
                        from_label2 = gr.Textbox(
                            value="From:",
                            text_align="right",
                            show_label=False,
                            container=False,
                            scale=1
                        )
                        from_field2 = gr.Textbox(
                            value="yina.arenas@woodgrovebank.com",
                            show_label=False,
                            container=False,
                            interactive=True,
                            scale=24
                        )
                    with gr.Row():
                        subject_label2 = gr.Textbox(
                            value="Subject:",
                            text_align="right",
                            show_label=False,
                            container=False,
                            scale=1
                        )
                        subject_field2 = gr.Textbox(
                            value="Re: Detailed Analysis Request",
                            show_label=False,
                            interactive=True,
                            container=False,
                            scale=24
                        )
                    # Email preview
                    # Initialize email_content by calling update_preview with initial values
                    initial_body = agent_response_email
                    with gr.Row():
                        # Define the update_preview function to accept all fields
                        def update_preview(body):
                            email_content = f"{body}"
                            # Find the image link in the body using Markdown image syntax
                            pattern = r'!\[.*?\]\((.*?)\)'
                            matches = re.findall(pattern, body)
                            if matches:
                                # Get the image path from the match
                                image_path_in_body = matches[0]
                                image_path = image_path_in_body.replace('file/', '')  # Remove 'file/' prefix if present
                                try:
                                    # Read the image file
                                    with open(image_path, 'rb') as img_file:
                                        img_data = img_file.read()
                                    # Encode the image to base64
                                    img_base64 = base64.b64encode(img_data).decode('utf-8')
                                    # Determine the image MIME type based on the file extension
                                    img_ext = image_path.split('.')[-1].lower()
                                    mime_type = f'image/{img_ext}' if img_ext in ['png', 'jpg', 'jpeg', 'gif'] else 'image/png'
                                    # Create the data URI
                                    data_uri = f"data:{mime_type};base64,{img_base64}"
                                    # Create an HTML <img> tag with desired size (e.g., width="600")
                                    html_img_tag = f'<img src="{data_uri}" alt="Tech Stock Performance" style="width:600px; height:auto;">'
                                    # Replace the Markdown image syntax in email_content with the HTML <img> tag
                                    email_content = re.sub(r'!\[.*?\]\(.*?\)', html_img_tag, email_content)
                                except Exception as e:
                                    print(f"Error reading image file: {e}")
                            return email_content

                        # Initialize email_preview with the initial content
                        initial_email_content = update_preview(initial_body)
                        # Email preview component
                        email_preview = gr.Markdown(value=initial_email_content, container=True, min_height=600, elem_classes=["body-textbox"])

                    # Simulate sending the email
                    def send_email():
                        time.sleep(2)  # Simulate sending email over 2 seconds
                        gr.Info("Email sent successfully to John Doe.")
                        # Return updates for email_column and components inside it
                        # Since we cannot update email_column directly, we'll update its child components
                        return [
                            gr.update(),             # Placeholder for email_column (no update needed)
                            gr.update(value=''),     # from_field2
                            gr.update(value=''),     # subject_field2
                            gr.update(value='')      # email_preview
                        ]

                    # Attach the send_email function to the btn_send click event
                    btn_send.click(
                        fn=send_email,
                        inputs=[],
                        outputs=[email_column, from_field2, subject_field2, email_preview],
                        show_progress="full"
                    )

        # Mount static files using FastAPI's StaticFiles
        demo.app.mount("/static", StaticFiles(directory="static"), name="static")

        # Return the demo app
        return demo

if __name__ == "__main__":
    demo = create_gradio_app()
    demo.launch(debug=True, allowed_paths=['/'])